In [ ]:

import numpy as np
import pandas as pd
import mysql.connector

In [2]:
df_ratin_final = pd.read_csv("Data/rating_final.csv")
df_accepts = pd.read_csv("Data/chefmozaccepts.csv")
df_cousine = pd.read_csv("Data/chefmozcuisine.csv")
df_houors4 = pd.read_csv("Data/chefmozhours4.csv")
df_geoplaces = pd.read_csv("Data/geoplaces2.csv")
df_parking = pd.read_csv("Data/chefmozparking.csv")

In [ ]:
lista_id = df_geoplaces["placeID"].unique()

df_accepts = df_accepts[df_accepts['placeID'].isin(lista_id)]
df_ratin_final = df_ratin_final[df_ratin_final['placeID'].isin(lista_id)]
df_cousine = df_cousine[df_cousine['placeID'].isin(lista_id)]
df_houors4 = df_houors4[df_houors4['placeID'].isin(lista_id)]
df_parking = df_parking[df_parking['placeID'].isin(lista_id)]

df_ratin_final.drop("userID", axis=1, inplace=True)

In [ ]:
lista_id = df_geoplaces["placeID"].unique()

df_accepts = df_accepts[df_accepts['placeID'].isin(lista_id)]
df_ratin_final = df_ratin_final[df_ratin_final['placeID'].isin(lista_id)]
df_cousine = df_cousine[df_cousine['placeID'].isin(lista_id)]
df_houors4 = df_houors4[df_houors4['placeID'].isin(lista_id)]
df_parking = df_parking[df_parking['placeID'].isin(lista_id)]

df_ratin_final.drop("userID", axis=1, inplace=True)

### FUNCIONES SQL (faltaría la contraseña de cada uno)

In [5]:
# ESTABLECER CONEXIÓN CON LA BASE DE DATOS DE MYSQL
# PARA GUARDAR LA CONEXIÓN Y EL CURSOR, HAY QUE IGUALAR LA FUNCIÓN A DOS VARIABLES
def conexion():
    db = mysql.connector.connect(host     = "localhost", # donde se aloja la base de datos
                             user     = "root", # poned vuestro usuario
                             password = "", # poned vuestra contraseña
                             database = None)
    cursor = db.cursor()
    return db, cursor

# CERRAR LA CONEXIÓN
def cerrar_conexion(cursor, db):
    cursor.fetchall() # se utiliza para recuperar todas las filas del conjunto de resultados después de ejecutar una consulta en una base de datos. Esto es útil cuando deseas trabajar con los resultados de una consulta en tu programa Python
    cursor.close()
    db.close()

# CREAR UNA BASE DE DATOS DANDOLE EL NOMBRE DE LA BASE DE DATOS
def crear_database(database):
    db, cursor = conexion()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database};")
    cerrar_conexion(cursor, db)

# CREAR UNA TABLA EN UNA BASE DE DATOS.
# PASANDOLE LA BASE DE DATOS DONDE QUEREMOS CREARLA Y LA QUERY DE LA TABLA QUE QUEREMOS CREAR
def crear_tablas(query, database):
    db, cursor = conexion()
    query_database = f"USE {database};"
    cursor.execute(query_database)
    cursor.execute(query)
    cerrar_conexion(cursor, db)

# EJECUTAR QUERYS DE CONSULTAS:
def ejecutar_consulta(database, query):
    db, cursor = conexion()
    cursor.execute(f"USE {database};")

    # Ejecutar la consulta
    cursor.execute(query)
    columnas = cursor.column_names
    # Obtener los resultados, si es una consulta SELECT
    resultados = cursor.fetchall()
    # Hacer commit para guardar los cambios (en caso de INSERT, UPDATE, DELETE, etc.)
    db.commit()
    # Cerrar la conexión
    cerrar_conexion(cursor, db)
    return resultados, columnas

def insertar_datos_porfila(df, database, tabla):
    db, cursor = conexion()
    cursor.execute(f"USE {database};")

    # Iterar sobre las filas del DataFrame e insertar cada fila en la tabla
    for _, fila in df.iterrows(): #Este método de pandas retorna un generador que produce pares (índice, fila) para cada fila del DataFrame df. 
                                  #el guion bajo _ se utiliza para descartar el índice, ya que no se está utilizando dentro del bucle.
                                  #Aquí el isinstance verifica que le valor es un str, sino lo convierte a str ya que la query tiene que ser str
        valores = ", ".join([f"'{valor}'" if isinstance(valor, str) else str(valor) for valor in fila])
        query = f"INSERT INTO {tabla} VALUES ({valores});"
        cursor.execute(query)

    # Hacer commit para guardar los cambios
    db.commit()
    cerrar_conexion(cursor, db)

In [ ]:
crear_database("recomendador")

In [ ]:
query_geoplaces = """CREATE TABLE IF NOT EXISTS geoplaces (
                    placeID INT UNSIGNED NOT NULL,
                    latitude FLOAT, 
                    longitude FLOAT, 
                    the_geom_meter VARCHAR(255), 
                    name VARCHAR(255), 
                    address VARCHAR(255),
                    city VARCHAR(255), 
                    state VARCHAR(255), 
                    country VARCHAR(255), 
                    fax VARCHAR(255), 
                    zip VARCHAR(255), 
                    alcohol VARCHAR(255), 
                    smoking_area VARCHAR(255),
                    dress_code VARCHAR(255), 
                    accessibility VARCHAR(255), 
                    price VARCHAR(255), 
                    url VARCHAR(255), 
                    Rambience VARCHAR(255), 
                    franchise VARCHAR(255),
                    area VARCHAR(255), 
                    other_services VARCHAR(255),
                    PRIMARY KEY (placeID)
                    );"""

query_payments = """CREATE TABLE IF NOT EXISTS payments (
                    placeID INT UNSIGNED NOT NULL,
                    Rpayment VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_cocina = """CREATE TABLE IF NOT EXISTS cocina (
                    placeID INT UNSIGNED NOT NULL,
                    Rcuisine VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_horario = """CREATE TABLE IF NOT EXISTS horario (
                    placeID INT UNSIGNED NOT NULL,
                    hours VARCHAR(255),
                    days VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_parking = """CREATE TABLE IF NOT EXISTS parking (
                    placeID INT UNSIGNED NOT NULL,
                    parking_lot VARCHAR(255),
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

query_rating = """CREATE TABLE IF NOT EXISTS rating (
                    placeID INT UNSIGNED NOT NULL,
                    rating INT UNSIGNED NOT NULL,
                    food_rating INT UNSIGNED NOT NULL,
                    service_rating INT UNSIGNED NOT NULL,
                    FOREIGN KEY (placeID) REFERENCES geoplaces (placeID)
                    );"""

### CREACIÓN TABLAS

In [ ]:
crear_tablas(query_geoplaces, "recomendador")
crear_tablas(query_payments, "recomendador")
crear_tablas(query_cocina, "recomendador")
crear_tablas(query_horario, "recomendador")
crear_tablas(query_parking, "recomendador")
crear_tablas(query_rating, "recomendador")

### INSERCCIÓN DATOS

In [ ]:
insertar_datos_porfila(df_geoplaces, "recomendador", "geoplaces")
insertar_datos_porfila(df_accepts, "recomendador", "payments")
insertar_datos_porfila(df_cousine, "recomendador", "cocina")
insertar_datos_porfila(df_houors4, "recomendador", "horario")
insertar_datos_porfila(df_parking, "recomendador", "parking")
insertar_datos_porfila(df_ratin_final, "recomendador", "rating")

In [3]:
query_select = """SELECT * FROM geoplaces
          JOIN payments ON geoplaces.placeID=payments.placeID
          JOIN cocina ON geoplaces.placeID=cocina.placeID
          JOIN horario ON geoplaces.placeID=horario.placeID
          JOIN parking ON geoplaces.placeID=parking.placeID
          ORDER BY geoplaces.placeID;
        """

In [6]:
resultado, columnas = ejecutar_consulta("recomendador", query_select)

In [7]:
df = pd.DataFrame(resultado, columns = columnas)

In [8]:
# Eliminar columnas duplicadas
df = df.loc[:, ~df.columns.duplicated()]

### TRATAMIENTO PARA TENER SOLO 1 VEZ EL RESTAURANTE POR FILA Y TENER TODAS SUS CARACTERISTICAS

In [9]:
df = df[["name", "alcohol", "dress_code", "accessibility", "smoking_area", "price", "Rambience", 'area', 'other_services', 'Rpayment', 'Rcuisine', 'hours', 'days', 'parking_lot']]

In [10]:
df.columns

Index(['name', 'alcohol', 'dress_code', 'accessibility', 'smoking_area',
       'price', 'Rambience', 'area', 'other_services', 'Rpayment', 'Rcuisine',
       'hours', 'days', 'parking_lot'],
      dtype='object')

In [11]:
columnas=['name', 'alcohol', 'dress_code', 'accessibility', 'smoking_area',
       'price', 'Rambience', 'area', 'other_services', 'Rpayment', 'Rcuisine',
        'parking_lot', 'Lunes_a_viernes', 'Sabado_Domingo']

In [12]:
df_recomendador=pd.DataFrame(columns=columnas)
df_recomendador

,name,alcohol,dress_code,accessibility,smoking_area,price,Rambience,area,other_services,Rpayment,Rcuisine,parking_lot,Lunes_a_viernes,Sabado_Domingo


In [13]:
nombres=set()
nombres=df.name.unique()

In [14]:


dic_caracteristicas = {}
primera_vez = True
nombre_anterior = ''

for indice, fila in df.iterrows():
    nombre_actual = fila['name']
    
    if nombre_actual != nombre_anterior:
        if not primera_vez:
            # Agregar diccionario al DataFrame existente usando concat
            df_recomendador = pd.concat([df_recomendador, pd.DataFrame([dic_caracteristicas])], ignore_index=True)

        # Reiniciar el diccionario y las variables de control
        dic_caracteristicas = {'name': nombre_actual,
                               'Rpayment': set(),
                               'Rcuisine': set(),
                               'Lunes_a_viernes': set(),
                               'Sabado_Domingo': set(),
                               'alcohol': fila['alcohol'],
                               'dress_code': fila['dress_code'],
                               'accessibility': fila['accessibility'],
                               'smoking_area': fila['smoking_area'],
                               'price': fila['price'],
                               'Rambience': fila['Rambience'],
                               'area': fila['area'],
                               'other_services': fila['other_services'],
                               'parking_lot': fila['parking_lot']}
        
        nombre_anterior = nombre_actual
        primera_vez = False

    # Resto del código para actualizar dic_caracteristicas
    dic_caracteristicas['Rpayment'].add(fila['Rpayment'])
    dic_caracteristicas['Rcuisine'].add(fila['Rcuisine'])
    if 'Mon' in fila['days']:
        dic_caracteristicas['Lunes_a_viernes'].add(fila['hours'])
    else:
        dic_caracteristicas['Sabado_Domingo'].add(fila['hours'])

# Al final del bucle, agregar la última entrada al DataFrame existente usando concat
df_recomendador = pd.concat([df_recomendador, pd.DataFrame([dic_caracteristicas])], ignore_index=True)

# Asegurarse de que las columnas tengan un tipo de datos adecuado (por ejemplo, str para objetos set)
df_recomendador = df_recomendador.applymap(lambda x: ', '.join(map(str, x)) if isinstance(x, set) else x)

In [15]:
df=df_recomendador.copy()
df_recomendador

,name,alcohol,dress_code,accessibility,smoking_area,price,Rambience,area,other_services,Rpayment,Rcuisine,parking_lot,Lunes_a_viernes,Sabado_Domingo
0,puesto de gorditas,No_Alcohol_Served,informal,no_accessibility,permitted,low,familiar,open,none,cash,Regional,public,08:00-12:00;,00:00-00:00;
1,Cafe Chaires,No_Alcohol_Served,informal,completely,not permitted,low,familiar,closed,none,cash,Cafeteria,yes,00:00-23:30;,00:00-23:30;
2,Gorditas Dona Tota,No_Alcohol_Served,informal,completely,not permitted,medium,familiar,closed,none,cash,Mexican,yes,08:30-20:00;,"09:00-20:30;, 09:00-22:00;"
3,tacos de barbacoa enfrente del Tec,No_Alcohol_Served,informal,completely,not permitted,low,familiar,open,none,cash,Mexican,public,09:30-23:30;,09:00-23:30;
4,Hamburguesas La perica,No_Alcohol_Served,informal,completely,permitted,low,quiet,open,none,cash,Mexican,public,19:00-02:00;,19:00-02:00;
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Mcdonalds Parque Tangamanga,No_Alcohol_Served,informal,no_accessibility,not permitted,medium,familiar,closed,none,"cash, MasterCard-Eurocard, VISA","Fast_Food, Burgers",yes,08:00-23:00;,08:00-23:00;
81,Cafeteria cenidet,No_Alcohol_Served,informal,no_accessibility,not permitted,low,quiet,closed,none,cash,Cafeteria,public,09:00-16:00;,00:00-00:00;
82,vips,Full_Bar,informal,completely,not permitted,medium,familiar,closed,variety,"cash, MasterCard-Eurocard, VISA",Mexican,yes,00:00-23:30;,00:00-23:30;
83,El Rinc�n de San Francisco,Wine-Beer,informal,partially,only at bar,medium,familiar,open,none,"cash, MasterCard-Eurocard, VISA",Mexican,none,18:00-23:30;,"18:00-23:30;, 18:00-21:00;"


### RECOMENDACION PERSONALIZADA PARTIENDO DE LA IDEA DE RECOMENDAR


- FUNCIONES DE APOYO

In [16]:
#FUNCION QUE RECIBE UNA LISTA QUE CONTIENE LOS DATOS SOBRE LOS RESTAURANTES Y SU PUNTUACION Y TE DEVUELVE
#DOS LISTAS ORDENADAS
def ordenamePeticion(lista)->list():
    nombresOrden=[]
    listaOrdenNum=[]
 
    for n,i in lista:
        listaOrdenNum.append(int(i))
    listaOrdenNum=sorted(listaOrdenNum,reverse=True)
    for i in listaOrdenNum:
        for a in lista:
            if i==a[1]:
                nombresOrden.append(a[0])
    # print(nombresOrden)  
    return listaOrdenNum,nombresOrden

#FUNCION QUE RECIVE UNA FILA EN FORMATO ARRAY , UN NUMERO QUE REPRESENTA LA PUNTUACION OTORGADA AL RESTAURANTE
#Y LAS CARACTERISTICAS DEL RESTAURANTE PARA REALIZAR LA PUNTUACION

def puntuacion(row, numero, caracteristicas):
    # Inicializa el contador de coincidencias
    contador_coincidencias = 0
    
    # Itera sobre las características y cuenta las coincidencias
    for caracteristica in caracteristicas[0]:
        # Verifica si la característica está presente en la fila
        # print(caracteristica)
        for c in row:
            
            if str(caracteristica) in str(c):
                contador_coincidencias+=1
        
        

    # Calcula la puntuación multiplicando el contador por el número
    puntaje = contador_coincidencias * numero
    return puntaje


In [17]:
#DATO DE EJEMPLO PARA REALIZAR LA RECOMENDACION
dato=[['puesto de gorditas',2],['Paniroles',8],['Pollo_Frito_Buenos_Aires',5]]

In [18]:
#FUNCION  PRINCIPAL QUE RECIBE EL DATAFRAME Y EL DATO QUE REPRESENTA A LAS PUNTUACIONES 
# Y TE DEVUELVE UN DATAFRAME CON LAS RECOMENDACIONES Y SU PUNTUACION
# EN ESTE CASO DEFINIMOS QUE POR CADA RESTAURANTE QUE SE LE PASE EN LA LISTA DE PUNTUACIOES
#DEVUELVA LOS 5 MAS PARECIDOS (LOS QUE TIENEN LA PUNTUACION MAS ALTA EN FUNCION A ESE RESTAURANTE)
def recomiendameOme(df, dato):
    total_caracteristicas = 11

    lista_puntuados, lista_nombres = ordenamePeticion(dato)

    df_recomendador = df.copy()

    # Quito los nombres del dataframe
    for n in dato:
        df_recomendador = df_recomendador[df_recomendador['name'] != n[0]]

    # Calcular la puntuación
    # Preparo los df para los resultados y las operaciones
    lista_columnas = df_recomendador.columns.to_list()
    lista_columnas.append('Puntuacion')
    
    df_resultado = pd.DataFrame(columns=lista_columnas)

    for n, i in zip(lista_nombres, lista_puntuados):
        caracteristicas = df[df['name'] == n].values
        # Recorrer el DataFrame por filas y calcular la puntuación
        puntuaciones = []

        for _, row in df_recomendador.iterrows():
            puntuacion_fila = puntuacion(np.array(row), i, caracteristicas)
            puntuaciones.append(puntuacion_fila)
            
        
        # Añadir la lista de puntuaciones como una nueva columna 'Puntuacion'
        
        df_recomendador['Puntuacion'] = puntuaciones
        
        # Ordenar el DataFrame por puntuación y seleccionar las mejores 5
        df_top5 = df_recomendador.sort_values('Puntuacion', ascending=False).head(5)
        
        # Concatenar los 5 mejores al DataFrame de resultados
        df_resultado = pd.concat([df_resultado, df_top5], ignore_index=True)
        
        # Restablecer el índice del DataFrame de resultados
        df_resultado.reset_index(drop=True, inplace=True)

    return df_resultado

In [19]:
resul=recomiendameOme(df,dato)

In [20]:
#IMPRIME EL RESULTADO
resul

,name,alcohol,dress_code,accessibility,smoking_area,price,Rambience,area,other_services,Rpayment,Rcuisine,parking_lot,Lunes_a_viernes,Sabado_Domingo,Puntuacion
0,Restaurant Orizatlan,Wine-Beer,informal,no_accessibility,none,medium,familiar,closed,none,"cash, MasterCard-Eurocard, VISA, American_Express",Mexican,none,08:00-22:00;,"08:00-23:00;, 08:00-19:00;",72
1,La Virreina,Wine-Beer,informal,no_accessibility,none,medium,familiar,closed,none,"cash, MasterCard-Eurocard, VISA, American_Express",Mexican,none,13:00-23:30;,"13:00-23:30;, 13:00-19:00;",72
2,Preambulo Wifi Zone Cafe,No_Alcohol_Served,informal,no_accessibility,none,medium,quiet,closed,Internet,cash,"Cafe-Coffee_Shop, Cafeteria",none,00:00-00:00;,00:00-00:00;,72
3,Luna Cafe,Wine-Beer,informal,no_accessibility,none,medium,familiar,closed,none,cash,"Bar, Cafeteria",none,00:00-00:00;,00:00-00:00;,72
4,Cenaduria El Rincón de Tlaquepaque,No_Alcohol_Served,informal,no_accessibility,none,medium,familiar,closed,none,cash,Mexican,none,19:30-23:30;,19:30-23:30;,64
5,tacos los volcanes,No_Alcohol_Served,informal,completely,none,low,quiet,closed,none,"MasterCard-Eurocard, American_Express, cash, b...",American,none,10:00-23:30;,10:00-23:30;,50
6,Cafe Chaires,No_Alcohol_Served,informal,completely,not permitted,low,familiar,closed,none,cash,Cafeteria,yes,00:00-23:30;,00:00-23:30;,45
7,little pizza Emilio Portes Gil,No_Alcohol_Served,informal,completely,none,low,familiar,closed,none,cash,Armenian,none,00:00-23:30;,00:00-23:30;,45
8,Hamburguesas Valle Dorado,No_Alcohol_Served,informal,completely,none,low,familiar,closed,none,cash,Burgers,none,18:30-23:30;,18:30-23:30;,45
9,KFC,No_Alcohol_Served,informal,completely,not permitted,low,familiar,closed,none,cash,"Game, American",yes,09:00-22:00;,09:00-22:00;,45
